In [2]:
import pandas as pd
import numpy as np

#Data comes from the U.S. Energy Information Administration
df = pd.read_csv("https://www.eia.gov/consumption/residential/data/2015/csv/recs2015_public_v4.csv")

print(df)

#KWH is probably the most useful target
Target = 'KWH'
print(df[Target].describe())

      DOEID  REGIONC  DIVISION  ... LPXBTU PERIODLP  ZLPAMOUNT
0     10001        4        10  ...  91.33       -2         -2
1     10002        3         7  ...  91.33       -2         -2
2     10003        3         6  ...  91.33       -2         -2
3     10004        2         4  ...  91.33        2          0
4     10005        1         2  ...  91.33       -2         -2
...     ...      ...       ...  ...    ...      ...        ...
5681  15682        2         3  ...  91.33       -2         -2
5682  15683        3         6  ...  91.33       -2         -2
5683  15684        2         3  ...  91.33       -2         -2
5684  15685        3         7  ...  91.33       -2         -2
5685  15686        2         3  ...  91.33       -2         -2

[5686 rows x 759 columns]
count     5686.000000
mean     11028.934872
std       7049.727589
min         59.078000
25%       5926.525750
50%       9549.351000
75%      14557.606750
max      63216.806000
Name: KWH, dtype: float64


In [4]:
#First, all imputation code columns are removed
df = df.loc[:, ~df.columns.str.startswith('Z')]

Census_Dict = {1:'New England', 2:'Middle Atlantic', 3:'East North Central', 4:'West North Central', 5:'South Atlantic', 6:'East South Central',7:'West South Central', 8:'Mountain North', 9:'Mountain South', 10:'Pacific'}


#Specify the list of housing features for model building. RECS Code information from https://www.eia.gov/consumption/residential/data/2015/xls/codebook_publicv4.xlsx
X_Features = ['REGIONC','TYPEHUQ','DISHWASH', 'CWASHER', 'DRYER', 'AIRCOND','NUMBERAC', 'NUMCFAN','NUMFLOORFAN','NUMWHOLEFAN','NUMATTICFAN','NOTMOIST','FUELH2O','USEEL','ELWARM','ELCOOL','ELWATER','ELFOOD','ELOTHER','TOTCSQFT','TOTHSQFT','TOTSQFT_EN','HEATHOME','TVCOLOR']

X = df[X_Features]

#Now replace an instance of negative numbers with 0 (i.e. no count of a feature)
X = X.clip(lower=0)
X = X.replace({"REGIONC": Census_Dict})

#For reference, list of features with present/not present indicators
X_Cat_Features = [i for i in X_Features if not i.startswith("NUM") | i.startswith("TVCOLOR") | i.startswith("TOT")]

#Encode region features
X = pd.get_dummies(X)
y = df['KWH']

#Here's the formatted features and target
print(X)
print(y)

      TYPEHUQ  DISHWASH  ...  REGIONC_New England  REGIONC_West North Central
0           2         1  ...                    0                           1
1           2         0  ...                    0                           0
2           2         1  ...                    0                           0
3           2         0  ...                    0                           0
4           2         0  ...                    1                           0
...       ...       ...  ...                  ...                         ...
5681        5         0  ...                    0                           0
5682        2         0  ...                    0                           0
5683        5         0  ...                    0                           0
5684        2         1  ...                    0                           0
5685        4         1  ...                    0                           0

[5686 rows x 27 columns]
0        5270.742
1       12173.000
2 

In [5]:
#Split data into train and test sets. Validation with the training set will be incorporated into the pipeline below
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#Model building with Keras MLP
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
# determine the number of input features
n_features = X_train.shape[1]
# define model
model = Sequential()
model.add(Dense(60, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(optimizer='adam', loss='MeanSquaredError')
# fit the model
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)

In [12]:
# evaluate the model
error_train = model.evaluate(X_train, y_train, verbose=0)
error = model.evaluate(X_test, y_test, verbose=0)

print('Training set MSE: %.3f' % error_train)
print('Training set RMSE: %.3f' % np.sqrt(error_train))
print('Test set MSE: %.3f' % error) 
print('Test set RMSE: %.3f' % np.sqrt(error))

#Now that error is below the standard deviation, it looks like we have a working model!

#View how the test set predictions and true values vary
print("Average Absolute Error:", round(np.average(np.abs(model.predict(X_test)-np.array(y_test).reshape(-1,1))),3))
print("Error Standard Deviation:", round(np.std(model.predict(X_test)-np.array(y_test).reshape(-1,1)),3))

#Uncomment the below to see the results!
#print(np.array(model.predict(X_test)).reshape(-1,1)[:5])
#print(np.array(y_test).reshape(-1,1)[:5])

Training set MSE: 26158166.000
Training set RMSE: 5114.505
Test set MSE: 25531752.000
Test set RMSE: 5052.895
Average Absolute Error: 3665.531
Error Standard Deviation: 5013.375
[[ 2379.0874]
 [12316.427 ]
 [15171.713 ]
 [ 8601.036 ]
 [20828.34  ]]
[[ 4601.811]
 [14049.246]
 [17559.995]
 [ 9244.033]
 [16000.457]]
